In [1]:
import pickle, os, easyocr
from pathlib import Path
reader = easyocr.Reader(['en'])

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


In [2]:
def remove_whitespace(line_lower):
    return  " ".join(line_lower.split())

In [3]:
from nltk.tokenize import RegexpTokenizer

def remove_punct(text):
    
    tokenizer = RegexpTokenizer(r"\w+")
    lst=tokenizer.tokenize(' '.join(text))
    return lst

In [4]:
from nltk.corpus import stopwords
en_stopwords = stopwords.words('english')

def remove_stopwords(text):
    result = []
    for token in text:
        if token not in en_stopwords:
            result.append(token)
            
    return result

In [5]:
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize,pos_tag

def lemmatization(text):
    
    result=[]
    wordnet = WordNetLemmatizer()
    for token,tag in pos_tag(text):
        pos=tag[0].lower()
        
        if pos not in ['a', 'r', 'n', 'v']:
            pos='n'
            
        result.append(wordnet.lemmatize(token,pos))
    
    return result

In [6]:
def image_to_text(image_path):


    if image_path.endswith('.png'):
        ocr_result = reader.readtext(str(image_path), batch_size=16)

        ocr_page = ""
        for bbox, word, confidance in ocr_result:
            ocr_page += word + " "

        with Path(str(image_path).replace('.png', '.txt')).open('w') as f:
            line_lower = ocr_page.lower()
            line_without_space = remove_whitespace(line_lower)
            token_list = word_tokenize(line_without_space)
            clean_text = remove_stopwords(token_list)
            no_punct = remove_punct(clean_text)
            lemma_text = lemmatization(no_punct)
            clean_text = " ".join(lemma_text)

            # json.dump(clean_text[1:-1], f)
            # text_file = open(os.path.join(text_folder, "data.txt"), 'w')
            f.write(clean_text)
            f.close()

In [7]:
loaded_model = pickle.load(open('model.1.0.0.pkl', 'rb'))

In [12]:
def predict_class(probability_array, prediction):

    ClassToBeReturned = "Cound not classify"
    if max(probability_array.tolist()) > 0.9:

        if prediction[0] == 1:
            ClassToBeReturned = 'W9 form'
        elif prediction[0] == 2:
            ClassToBeReturned = 'iBAN'
        elif prediction[0] == 3:
            ClassToBeReturned = 'ISO certificate'


    return ClassToBeReturned

In [13]:
path = os.path.join(os.getcwd(), 'Test Images')
image_path = os.path.join(path, 'W9-200.png')
text_path = image_path.replace('.png', '.txt')

In [14]:
image_to_text(image_path)

### The Predict method predicts 
1: W9 | 2: iBAN | 3: ISO


In [16]:
%%time
with open(text_path, 'r') as f:
    text = f.readlines()
    probability_array = loaded_model.predict_proba(text)[0]
    prediction = loaded_model.predict(text)
    predicted_class = predict_class(probability_array, prediction)
    print("The Probability array: ", probability_array)
    print("Prediction: ", prediction)
    print('Predicted Class: ', predicted_class)

The Probability array:  [1. 0. 0.]
Prediction:  [1]
Predicted Class:  W9 form
Wall time: 24.4 ms
